### Llama 2 13B Chat

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation in Linux and with Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex
from llama_index import SimpleDirectoryReader
from llama_index import ServiceContext

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [4]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/llama-2-13b-chat.Q6_K.gguf',
    temperature=0.1,
    max_new_tokens=1024, # Increasing to support longer responses
    context_window=4096, # Llama 2 4096 context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 41}, # 41 was all that was needed and fits within the RTX 3090's memory
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from ./Models/llama-2-13b-chat.Q6_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tenso

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large", cache_folder=None)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Did not find folder thenlper/gte-large
Did not find folder thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Search model on server: http://sbert.net/models/thenlper/gte-large.zip
Search model on server: http://sbert.net/models/thenlper/gte-large.zip
INFO:sentence_transformers.SentenceTransformer:Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


#### 6. Prompt Template

Prompt template for Llama2 - see: https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ/discussions/5#64b8e6cdf8bf823a61ed1243

The function creates a prompt given a dictionary of role + message entries (roles are assistant, which is the LLM, and user, which is your questions)

In [7]:
# Produces a prompt for the Llama2 model
def llama_v2_prompt(
    messages: list[dict]
):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    DEFAULT_SYSTEM_PROMPT = f"""You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages
    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]

    messages_list = [
        f"{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} "
        for prompt, answer in zip(messages[::2], messages[1::2])
    ]
    messages_list.append(f"{B_INST} {(messages[-1]['content']).strip()} {E_INST}")

    return "".join(messages_list)

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [8]:
service_context = ServiceContext.from_defaults(
    chunk_size=256, # Number of tokens in each chunk
    llm=llm,
    embed_model=embed_model
)

#### 8. Index documents

In [9]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [10]:
from llama_index.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [11]:
def RunQuestion(questionText):
    llama2Prompt = [
    {"role": "system", "content": "You are a technology specialist. Answer questions in a positive, helpful and empathetic way. If the answer is not in the following context return ONLY 'Sorry, I don't know the answer to that'."},
    {"role": "user", "content": f"{questionText}"}
]
    queryQuestion = llama_v2_prompt(llama2Prompt)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [12]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [13]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =    3159.39 ms
llama_print_timings:      sample time =      88.46 ms /   324 runs   (    0.27 ms per token,  3662.80 tokens per second)
llama_print_timings: prompt eval time =   10910.29 ms /  1358 tokens (    8.03 ms per token,   124.47 tokens per second)
llama_print_timings:        eval time =   53879.36 ms /   323 runs   (  166.81 ms per token,     5.99 tokens per second)
llama_print_timings:       total time =   65535.58 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


llama_print_timings:        load time =    3159.39 ms
llama_print_timings:      sample time =      39.67 ms /   151 runs   (    0.26 ms per token,  3806.50 tokens per second)
llama_print_timings: prompt eval time =    8449.12 ms /   992 tokens (    8.52 ms per token,   117.41 tokens per second)
llama_print_timings:        eval time =   23185.95 ms /   150 runs   (  154.57 ms per token,     6.47 tokens per second)
llama_print_timings:       total time =   31967.57 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =    3159.39 ms
llama_print_timings:      sample time =      79.45 ms /   292 runs   (    0.27 ms per token,  3675.04 tokens per second)
llama_print_timings: prompt eval time =    9482.91 ms /  1066 tokens (    8.90 ms per token,   112.41 tokens per second)
llama_print_timings:        eval time =   47434.38 ms /   291 runs   (  163.00 ms per token,     6.13 tokens per second)
llama_print_timings:       total time =   57583.21 ms


#### 13. Output responses

In [14]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

Certainly! Here's a summary of the story based on the provided sources:

In the future, a prehistoric dinosaur named Thundertooth traveled through time and arrived in a futuristic city. The city's mayor, Grace, welcomed him with open arms and offered to help find a solution to his hunger dilemma. Together, they explored the city's innovative food labs and discovered a sustainable solution that satisfied Thundertooth's hunger without harming anyone. As the news of Thundertooth's arrival spread, the city embraced him as a symbol of unity between the past and the future. When a massive meteor threatened the city, Thundertooth and the mayor worked together to find a plan to divert or neutralize the threat.

Source 1: "Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.

Source 2: "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone."

Sou